In [3]:
import numpy as np
import pandas as pd

import pickle
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.base import TransformerMixin, BaseEstimator

Шаг 1. Обучение модели

In [4]:
# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

LinearRegression()

Шаг 2. Сериализация

In [5]:
# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


In [6]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

Шаг 3. Запись модели в файл

In [7]:
# Производим сериализацию и записываем результат в файл формата pkl
#with open('myfile.pkl', 'wb') as output:
    #pickle.dump(regressor, output)

Шаг 5. Загружаем модель из файла

In [8]:
# Производим десериализацию и извлекаем модель из файла формата pkl
#with open('myfile.pkl', 'rb') as pkl_file:
    #regressor_from_file = pickle.load(pkl_file)

#regressor_from_file

Использование пайплайнов

In [9]:
# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000202C10084C0>)),
                ('Linear', LinearRegression())])

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами sklearn. Например, для решения многих задач в нашем курсе мы часто использовали feature engineering, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в sklearn можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: TransformerMixin и BaseEstimator.

In [10]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

In [11]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

Shape before transform: (442, 10)
Shape after transform: (442, 11)


In [12]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000202C10084C0>)),
                ('Linear', LinearRegression())])

In [13]:
features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])
pipe.predict(features)

array([173.01985747])

Использование библиотеки Joblib

In [14]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
#joblib.dump(regressor, 'regr.joblib')

LinearRegression()

In [15]:
# Десериализуем модель из файла
#clf_from_jobliv = joblib.load('regr.joblib') 

PMML — это XML-диалект, который применяется для описания статистических и DS-моделей. PMML-совместимые приложения позволяют легко обмениваться моделями данных между собой.

In [16]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
#skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

Pipeline(steps=[('Scaling', MinMaxScaler()), ('Linear', LinearRegression())])

Обучаем модель sklearn, конвертируем ее в ONNX и делаем инференс через ONNX-runtime.

In [17]:
import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', FloatTensorType([None, 13]))]
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("data/model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.InferenceSession("data/model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)

(379, 13) (127, 13)
sklearn model predict:
 [23.1903541  18.97985889 19.82548836 19.00126197  4.39524325 11.90230303
 21.24870187 28.64449553 29.03550064 13.90644782  6.41422339 32.65356658
 18.99884691 20.01569489 37.15275422 22.80485488 29.04529555 33.04200949
 10.48602033 24.45472284 21.33069324 27.60222354 37.52118276 13.6113556
  9.56442243 15.03368415 35.5975585  26.01017573 25.52430154 27.06321433
 19.07680237 30.54746571 31.27561168 16.40132981 39.76707419 20.27263903
 18.94934061 17.12210014 21.6262832  28.15101424 26.95292863 19.14352801
 14.50664721 25.78075705 18.50460146 13.93439214 24.96593139 19.12431756
 20.6780475   6.23807397 27.71460362 26.74617711 11.83361779 40.10855118
 14.66523328 22.12023896 20.34305401 20.3786179  23.56685605 21.91582872
 20.79748126 35.43123681 17.32592458 20.92077502 24.1674162  43.38199388
 19.59747681 20.11624895 22.35462757 28.12506906 25.53832602 12.88949504
 13.1552648  33.3092473  26.12666965 22.54135443 12.14404271 16.61972119
 28.5270

c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets incl